In [1]:
import ast
from collections import Counter
from time import sleep
from urllib.request import Request, urlopen

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
df = pd.concat([pd.read_csv("initial_dataset/train.csv")[["Drug_ID", "Protein_ID"]], \
                pd.read_csv("initial_dataset/val.csv")[["Drug_ID", "Protein_ID"]], \
                pd.read_csv("initial_dataset/test.csv")[["Drug_ID", "Protein_ID"]]])

In [3]:
cID = np.array(list(Counter(np.concatenate((df["Drug_ID"].values, df["Protein_ID"].values))).keys()))
DB_ID = [x for x in cID if x[0] == "D"]
prot_ID = [x for x in cID if x[0] != "D"][1:]
len(prot_ID), len(DB_ID)

(2325, 5017)

### Protein ID to Sequnce

In [4]:
# %%time

# seq = {}

# baseUrl = "http://www.uniprot.org/uniprot/"
# maxTries = 3
# for i in tqdm(prot_ID):
#     currentUrl = baseUrl + i + ".fasta"
#     delay = 5
#     for _ in range(maxTries):
#         try:
#             page = urlopen(currentUrl)
#             html = page.read().decode("utf-8")
#             soup = BeautifulSoup(html, "html.parser")
#             seq[i] = soup.text
#             break
#         except:
#             sleep(delay)
#             delay *= 2
            
# for i in seq.keys():
#     seq[i] = ''.join(seq[i].split("\n")[1:])

In [5]:
# # save prot_ID
# file = open('prot_ID.txt', 'wt')
# file.write(str(seq))
# file.close()

In [6]:
# # load prot_ID
file = open('prot_ID.txt', 'r')
seq = ast.literal_eval(file.read())

### Durg ID to Chemical Formula

In [7]:
db_seq = {}
notFetched = []

In [8]:
# %%time

# baseUrl = "https://go.drugbank.com/drugs/"

# for i in (DB_ID):
#     try:
#         url = baseUrl + i
#         req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
#         page = urlopen(req)
#         html = page.read().decode("utf-8")
#         soup = BeautifulSoup(html, "html.parser")

#         d = soup.find_all("div", attrs={"class": "wrap"})
#         db_seq[i] = d[2].text
#     except:
#         notFetched.append(i)
#         continue

In [9]:
# # save drug_ID
# file = open('drug_ID.txt', 'wt')
# file.write(str(db_seq))
# file.close()

In [10]:
# # load drug_ID
file = open('drug_ID.txt', 'r')
db_seq = ast.literal_eval(file.read())

### Replacing ID

In [11]:
df = pd.concat([pd.read_csv("initial_dataset/train.csv")[["Drug_ID", "Protein_ID", "label"]], 
                pd.read_csv("initial_dataset/val.csv")[["Drug_ID", "Protein_ID", "label"]], 
                pd.read_csv("initial_dataset/test.csv")[["Drug_ID", "Protein_ID", "label"]]])

In [12]:
df = df[df["Drug_ID"].isin(set(list(db_seq.keys())))]

In [13]:
df = df[df["Protein_ID"].isin(set(list(seq.keys())))]

In [14]:
newdf = pd.DataFrame(columns=["Drug", "Protien", "label"]) 
newdf["Drug"] = df["Drug_ID"].apply(lambda x: db_seq[x])
newdf["Protien"] = df["Protein_ID"].apply(lambda x: seq[x])
newdf["label"] = df["label"]

In [15]:
newdf.head()

,Drug,Protien,label
0,IC1=CNN=C1,MSTAGKVIKCKAAVLWEVKKPFSIEDVEVAPPKAYEVRIKMVAVGI...,1
1,CC(=O)NC1=CNN=C1C(=O)NC1=CC=C(F)C=C1,MENFQKVEKIGEGTYGVVYKARNKLTGEVVALKKIRLDTETEGVPS...,1
2,CN(C)CCCN1C2=CC=CC=C2CCC2=C1C=C(Cl)C=C2,MLLARMNPQVQPENNGADTGPEQPLRARKTAELLVVKERNGVQCLL...,1
3,ClC1=CC=CC(N2CCN(CCCCOC3=CC4=C(CCC(=O)N4)C=C3)...,MANFTPVNGSSGNQSVRLVTSSSHNRYETVEMVFIATVTGSLSLVT...,1
4,OC1N=C(C2=CC=CC=C2Cl)C2=C(NC1=O)C=CC(Cl)=C2,MVSAKKVPAIALSAGVSFALLRFLCLAVCLNESPGQNQKEEKLCTE...,1


In [16]:
newdf.to_csv("preprocessed_data.csv", index=False)